In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.context import context

from src.utilities import *
from src.bubble import *

import instructor



Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [2]:
aspects_evol_df = get("Aspect Evolution")

In [4]:
df = aspects_evol_df[aspects_evol_df["Period"] == 'M']
df[["Date", "Category", "Mean Rating"]]


,Date,Category,Mean Rating
10,2023-03-31 00:00:00+00:00,1708015570991x523003420569127600,5.000000
11,2023-06-30 00:00:00+00:00,1708015553720x307677447830442240,4.000000
12,2023-07-31 00:00:00+00:00,1708015569716x598042261554836900,1.000000
13,2023-07-31 00:00:00+00:00,1708015570991x523003420569127600,1.000000
14,2023-08-31 00:00:00+00:00,1708015553720x307677447830442240,1.000000
...,...,...,...
215,2024-02-29 00:00:00+00:00,NaN,1.000000
216,2024-02-29 00:00:00+00:00,NaN,3.000000
217,2024-02-29 00:00:00+00:00,NaN,2.777778
218,2024-02-29 00:00:00+00:00,NaN,1.000000


In [5]:
import pandas as pd

# Create your DataFrame
df = pd.DataFrame({'Date': ['2023-06-30 00:00:00+00:00', '2023-06-30 00:00:00+00:00',
                           '2023-07-31 00:00:00+00:00', '2023-07-31 00:00:00+00:00',
                           '2023-08-31 00:00:00+00:00'],
                   'Category': ['1708015570991x523003420569127600', '1708015553720x307677447830442240',
                               '1708015569716x598042261554836900', '1708015570991x523003420569127600',
                               '1708015553720x307677447830442240'],
                   'Count': [5.0, 4.0, 1.0, 1.0, 1.0]})

# Convert 'Date' to datetime format and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(['Date', 'Category'], inplace=True)

# Resample to monthly frequency and fill missing values with 0
result = df.unstack(fill_value=0).asfreq('M', fill_value=0)

count_df = result["Count"]
count_df

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/1152064990.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])


ValueError: Index contains duplicate entries, cannot reshape

In [7]:
df = aspects_evol_df

In [43]:
import pandas as pd

# Create your DataFrame
df = pd.DataFrame({'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-02-01', '2023-02-02', '2023-02-03', '2023-03-01'],
                   'Category': ['A', 'B', 'A', 'C', 'A', 'B', 'A']})

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Resample to monthly frequency and count occurrences of each category
result = df.resample('M', on='Date')['Category'].value_counts().unstack(fill_value=0)

# Print the result
result


/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/3122372447.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample('M', on='Date')['Category'].value_counts().unstack(fill_value=0)


Category,A,B,C
Date,,,
2023-01-31,2,1,0
2023-02-28,1,1,1
2023-03-31,1,0,0


In [9]:
df = aspects_evol_df[['Date', 'Category', 'SubCategory']]
df.fillna('', inplace=True)

df

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/4168413704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna('', inplace=True)


,Date,Category,SubCategory
0,2023-12-31 00:00:00+00:00,1708015553720x307677447830442240,
1,2023-12-31 00:00:00+00:00,1708015555422x783692141072623600,
2,2023-12-31 00:00:00+00:00,1708015569716x598042261554836900,
3,2023-12-31 00:00:00+00:00,1708015570991x523003420569127600,
4,2023-12-31 00:00:00+00:00,1708015577252x623767797641778300,
...,...,...,...
374,2024-02-07 00:00:00+00:00,,1708015554529x444035970008549250
375,2024-02-07 00:00:00+00:00,,1708015555741x283185418299470850
376,2024-02-07 00:00:00+00:00,,1708015570395x458495376435047900
377,2024-02-07 00:00:00+00:00,,1708015570687x524341896218553100


In [10]:
result = df.resample('M', on='Date')[['Category', 'SubCategory']].value_counts().unstack(fill_value=0)
result

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/1307452999.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample('M', on='Date')[['Category', 'SubCategory']].value_counts().unstack(fill_value=0)


SubCategory                                                     \
Date                      Category                               
2023-03-31 00:00:00+00:00                                    0   
                          1708015570991x523003420569127600   2   
2023-06-30 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   2   
2023-07-31 00:00:00+00:00                                    0   
                          1708015569716x598042261554836900   2   
                          1708015570991x523003420569127600   2   
2023-08-31 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   2   
                          1708015570991x523003420569127600   2   
2023-10-31 00:00:00+00:00                                    0   
                          1708015570991x523003420569127600   2   
2023-11-30 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   2   
                          1708015570991x523003420569127600   2   
2023-12-31 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   3   
                          1708015555422x783692141072623600   3   
                          1708015569716x598042261554836900   5   
                          1708015570991x523003420569127600  10   
                          1708015577252x623767797641778300   7   
2024-01-31 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240  20   
                          1708015555422x783692141072623600  12   
                          1708015569716x598042261554836900  15   
                          1708015570991x523003420569127600  24   
                          1708015577252x623767797641778300   8   
2024-02-29 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   5   
                          1708015555422x783692141072623600   3   
                          1708015569716x598042261554836900   6   
                          1708015570991x523003420569127600   7   
2024-12-31 00:00:00+00:00                                    0   
                          1708015553720x307677447830442240   1   
                          1708015555422x783692141072623600   1   
                          1708015569716x598042261554836900   1   
                          1708015570991x523003420569127600   1   
                          1708015577252x623767797641778300   1   

SubCategory                                                 1708015554131x995344137291493100  \
Date                      Category                                                             
2023-03-31 00:00:00+00:00                                                                  0   
                          1708015570991x523003420569127600                                 0   
2023-06-30 00:00:00+00:00                                                                  2   
                          1708015553720x307677447830442240                                 0   
2023-07-31 00:00:00+00:00                                                                  0   
                          1708015569716x598042261554836900                                 0   
                          1708015570991x523003420569127600                                 0   
2023-08-31 00:00:00+00:00                                                                  0   
                          1708015553720x307677447830442240                                 0   
                          1708015570991x523003420569127600                                 0   
2023-10-31 00:00:00+00:00                                                                  0   
                          1708015570991x523003420569127600                                 0   
2023-11-30 00:0

In [14]:
result.columns

Index(['', '1708015554131x995344137291493100',
       '1708015554529x444035970008549250', '1708015555741x283185418299470850',
       '1708015568900x728837610303776400', '1708015569398x419300495562577400',
       '1708015570097x862705649430030700', '1708015570395x458495376435047900',
       '1708015570687x524341896218553100', '1708015571499x209656851322028060',
       '1708015573399x329923182630295500', '1708015576620x263266380365956300',
       '1708015577555x688807197740501000', '1708015577923x774632487944991200',
       '1708015580113x761173827728466800'],
      dtype='object', name='SubCategory')

In [13]:
result.index

MultiIndex([('2023-03-31 00:00:00+00:00',                                 ''),
            ('2023-03-31 00:00:00+00:00', '1708015570991x523003420569127600'),
            ('2023-06-30 00:00:00+00:00',                                 ''),
            ('2023-06-30 00:00:00+00:00', '1708015553720x307677447830442240'),
            ('2023-07-31 00:00:00+00:00',                                 ''),
            ('2023-07-31 00:00:00+00:00', '1708015569716x598042261554836900'),
            ('2023-07-31 00:00:00+00:00', '1708015570991x523003420569127600'),
            ('2023-08-31 00:00:00+00:00',                                 ''),
            ('2023-08-31 00:00:00+00:00', '1708015553720x307677447830442240'),
            ('2023-08-31 00:00:00+00:00', '1708015570991x523003420569127600'),
            ('2023-10-31 00:00:00+00:00',                                 ''),
            ('2023-10-31 00:00:00+00:00', '1708015570991x523003420569127600'),
            ('2023-11-30 00:00:00+00:00',           

In [24]:
df.values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

In [26]:
for (date, cat), s in result.iterrows():
    print(date)
    print(cat)
    for sub_cat, count in zip(s.index, s.values):
        print(sub_cat, count)

2023-03-31 00:00:00+00:00

 0
1708015554131x995344137291493100 0
1708015554529x444035970008549250 0
1708015555741x283185418299470850 0
1708015568900x728837610303776400 0
1708015569398x419300495562577400 0
1708015570097x862705649430030700 0
1708015570395x458495376435047900 0
1708015570687x524341896218553100 0
1708015571499x209656851322028060 2
1708015573399x329923182630295500 0
1708015576620x263266380365956300 0
1708015577555x688807197740501000 0
1708015577923x774632487944991200 0
1708015580113x761173827728466800 0
2023-03-31 00:00:00+00:00
1708015570991x523003420569127600
 2
1708015554131x995344137291493100 0
1708015554529x444035970008549250 0
1708015555741x283185418299470850 0
1708015568900x728837610303776400 0
1708015569398x419300495562577400 0
1708015570097x862705649430030700 0
1708015570395x458495376435047900 0
1708015570687x524341896218553100 0
1708015571499x209656851322028060 0
1708015573399x329923182630295500 0
1708015576620x263266380365956300 0
1708015577555x688807197740501000 

In [34]:
dicts[-1]

{'Company': '1707313014508x102198350946437700',
 'Count': 0,
 'Date': '2024-12-31 00:00:00+00:00',
 'Location': 'FR',
 'Project': '1707329196900x870734705097005300',
 'Source': 'Trustpilot',
 'Category': '1708015577252x623767797641778300',
 'SubCategory': '1708015580113x761173827728466800'}

In [37]:
categories_df = get("Category")

,Created Date,Created By,Modified Date,Company,Name,Project,_id
0,2024-02-15 16:45:53.720000+00:00,admin_user_feedback-analysis_test,2024-02-15 16:45:53.721000+00:00,1707313014508x102198350946437700,Recherche de Produit,1707329196900x870734705097005300,1708015553720x307677447830442240
1,2024-02-15 16:45:55.422000+00:00,admin_user_feedback-analysis_test,2024-02-15 16:45:55.422000+00:00,1707313014508x102198350946437700,Processus d'Achat,1707329196900x870734705097005300,1708015555422x783692141072623600
2,2024-02-15 16:46:09.716000+00:00,admin_user_feedback-analysis_test,2024-02-15 16:46:09.716000+00:00,1707313014508x102198350946437700,Livraison,1707329196900x870734705097005300,1708015569716x598042261554836900
3,2024-02-15 16:46:10.991000+00:00,admin_user_feedback-analysis_test,2024-02-15 16:46:10.991000+00:00,1707313014508x102198350946437700,Service Après-Vente,1707329196900x870734705097005300,1708015570991x523003420569127600
4,2024-02-15 16:46:17.252000+00:00,admin_user_feedback-analysis_test,2024-02-15 16:46:17.252000+00:00,1707313014508x102198350946437700,Expérience en Magasin,1707329196900x870734705097005300,1708015577252x623767797641778300


In [36]:
batchs = batchify(result, 1000)
for batch in tqdm(batchs):
    dicts = []
    for (date, cat), s in result.iterrows():
        for sub_cat, count in zip(s.index, s.values):
            dicts.append({
                "Company": COMPANY_ID,
                "Count": float(count),
                "Date": str(date),
                "Location": 'FR',
                "Project": PROJECT_ID,
                "Source": "Trustpilot",
                "Category": cat, #if cat!='' else None,
                "SubCategory": sub_cat, #if sub_cat!='' else None,
            })

    bubble_client.create("Aspect Evol",dicts)

0it [00:00, ?it/s]

In [ ]:
row_label = 'your_desired_row_label'
category = df.loc[row_label, 'Category']
print(category)

In [64]:
i, row = next(result.iterrows())
i

(Timestamp('2023-03-31 00:00:00+0000', tz='UTC'), '')

In [58]:
result.iloc[1]

SubCategory
                                    2
1708015554131x995344137291493100    0
1708015554529x444035970008549250    0
1708015555741x283185418299470850    0
1708015568900x728837610303776400    0
1708015569398x419300495562577400    0
1708015570097x862705649430030700    0
1708015570395x458495376435047900    0
1708015570687x524341896218553100    0
1708015571499x209656851322028060    0
1708015573399x329923182630295500    0
1708015576620x263266380365956300    0
1708015577555x688807197740501000    0
1708015577923x774632487944991200    0
1708015580113x761173827728466800    0
Name: (2023-03-31 00:00:00+00:00, 1708015570991x523003420569127600), dtype: int64

In [56]:
result['1708015554131x995344137291493100']

Date                       Category                        
2023-03-31 00:00:00+00:00                                      0
                           1708015570991x523003420569127600    0
2023-06-30 00:00:00+00:00                                      2
                           1708015553720x307677447830442240    0
2023-07-31 00:00:00+00:00                                      0
                           1708015569716x598042261554836900    0
                           1708015570991x523003420569127600    0
2023-08-31 00:00:00+00:00                                      0
                           1708015553720x307677447830442240    0
                           1708015570991x523003420569127600    0
2023-10-31 00:00:00+00:00                                      0
                           1708015570991x523003420569127600    0
2023-11-30 00:00:00+00:00                                      0
                           1708015553720x307677447830442240    0
                           170

In [ ]:

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])


In [ ]:
[['Category', 'SubCategory']]

In [41]:
import pandas as pd

# Your DataFrame
df = pd.DataFrame({
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-02-01', '2023-02-02', '2023-02-03', '2023-03-01'],
    '_id': [1, 2, 3, 4, 5, 6, 7],
    'Category': ['A', 'B', 'A', 'C', 'A', 'B', 'A'],
    'SubCategory': ['X', 'Y', 'X', 'Z', 'X', 'Y', 'X']
})

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])


In [42]:
result = df.resample('M', on='Date')[['Category', 'SubCategory']].value_counts().unstack(fill_value=0)
result

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/1307452999.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample('M', on='Date')[['Category', 'SubCategory']].value_counts().unstack(fill_value=0)


SubCategory          X  Y  Z
Date       Category         
2023-01-31 A         2  0  0
           B         0  1  0
2023-02-28 A         1  0  0
           B         0  1  0
           C         0  0  1
2023-03-31 A         1  0  0

In [18]:
_,row = next(df.iterrows())


4

In [19]:
for i, row in df.iterrows():
    bubble_client.update_object(bubble_type="Aspect Evolution", bubble_id=row["_id"], fields={"Count": sum([row["Count of "+str(k)+"s"] for k in range(1,6)])})


In [53]:
data = {'date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-02-01', '2023-02-02', '2023-02-03'],
        'category': ['A', 'A', 'B', 'A', 'C', 'B', 'C'],
        'subcategory': ['a', 'b', 'a', 'a', None, 'b', None],
        'count': [1, 2, 3, 4, 5, 6, 7]}
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])

groups = df.groupby([df.date.dt.year, df.date.dt.month, 'category', 'subcategory'])
result = groups['count'].agg('sum').unstack(fill_value=0)
result


,date,category,subcategory,count
0,2023-01-01,A,a,1
1,2023-01-02,A,b,2
2,2023-01-03,B,a,3
3,2023-01-04,A,a,4
4,2023-02-01,C,None,5
5,2023-02-02,B,b,6
6,2023-02-03,C,None,7


In [54]:
df['date'].iloc[0]

Timestamp('2023-01-01 00:00:00')

In [59]:
groups = df.groupby([df.date.dt.year, df.date.dt.month, 'category', 'subcategory'])
result = groups['count'].agg('sum').unstack(fill_value=0)
result

subcategory         a  b
date date category      
2023 1    A         5  2
          B         3  0
     2    B         0  6

In [64]:
import pandas as pd

data = {'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-02-01', '2023-02-02', '2023-02-03'],
        'Category': ['A', 'A', 'B', 'A', 'C', 'B', 'C'],
        'SubCategory': ['a', 'b', 'c', 'a', None, 'c', None],
        'Count': [1, 2, 3, 4, 5, 6, 7]}
df = pd.DataFrame(data)

def g(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by=['Date', 'Category', 'SubCategory'], inplace=True)
    df['Date'] = df['Date'].dt.strftime('%Y-%m')  # Format as 'YYYY-MM'
    return df.pivot_table(index='Date', columns=['Category', 'SubCategory'], values='Count', aggfunc=sum).fillna(0)

result = g(df.copy())
print(result)


Category       A         B
SubCategory    a    b    c
Date                      
2023-01      5.0  2.0  3.0
2023-02      0.0  0.0  6.0


/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/526220156.py:13: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  return df.pivot_table(index='Date', columns=['Category', 'SubCategory'], values='Count', aggfunc=sum).fillna(0)


In [69]:
aspects_df = get("Aspect")

In [72]:
aspects_df["Count"] = 1

In [89]:
results = g(aspects_df[["Count", "Date", "Category", "SubCategory"]])
results

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/526220156.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/526220156.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=['Date', 'Category', 'SubCategory'], inplace=True)
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_9703/526220156.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

Category    1708015553720x307677447830442240                                   \
SubCategory 1708015554131x995344137291493100 1708015554529x444035970008549250   
Date                                                                            
2023-03                                  0.0                              0.0   
2023-06                                  1.0                              0.0   
2023-07                                  0.0                              0.0   
2023-08                                  0.0                              1.0   
2023-10                                  0.0                              0.0   
2023-11                                  0.0                              1.0   
2023-12                                  1.0                              0.0   
2024-01                                  8.0                             24.0   
2024-02                                  1.0                             10.0   

Category    1708015555422x783692141072623600                                   \
SubCategory 1708015555741x283185418299470850 1708015568900x728837610303776400   
Date                                                                            
2023-03                                  0.0                              0.0   
2023-06                                  0.0                              0.0   
2023-07                                  0.0                              0.0   
2023-08                                  0.0                              0.0   
2023-10                                  0.0                              0.0   
2023-11                                  0.0                              0.0   
2023-12                                  0.0                              0.0   
2024-01                                  6.0                              2.0   
2024-02                                  2.0                              0.0   

Category                                     1708015569716x598042261554836900  \
SubCategory 1708015569398x419300495562577400 1708015570097x862705649430030700   
Date                                                                            
2023-03                                  0.0                              0.0   
2023-06                                  0.0                              0.0   
2023-07                                  0.0                              1.0   
2023-08                                  0.0                              0.0   
2023-10                                  0.0                              0.0   
2023-11                                  0.0                              0.0   
2023-12                                  1.0                              2.0   
2024-01                                  5.0                              5.0   
2024-02                                  0.0                              2.0   

Category                                                                       \
SubCategory 1708015570395x458495376435047900 1708015570687x524341896218553100   
Date                                                                            
2023-03                                  0.0                              0.0   
2023-06                                  0.0                              0.0   
2023-07                                  0.0                              0.0   
2023-08                                  0.0                              0.0   
2023-10                                  0.0                              0.0   
2023-11                                  0.0                              0.0   
2023-12                                  1.0                              1.0   
2024-01                                  1.0                             14.0   
2024-02                                  2.0                              6.0   

Category    1708015570991x523003420569127600                                   \
SubCategory 1708015571499x2

In [92]:
batchs = batchify(results, 1000)
for batch in tqdm(batchs):
    dicts = []
    for date, serie in batch.iterrows():
        for (cat, sub_cat), count in zip(serie.index, serie.values):
            dicts.append({
                "Company": COMPANY_ID,
                "Count": float(count),
                "Date": str(date),
                "Location": 'FR',
                "Project": PROJECT_ID,
                "Source": "Trustpilot",
                "Category": cat, #if cat!='' else None,
                "SubCategory": sub_cat, #if sub_cat!='' else None,
            })

    bubble_client.create("Aspect Evol",dicts)

0it [00:00, ?it/s]